This is the DutchessV1 Script, you should run the assistant creator and resource creator scripts before you run this one. You should also check that within the same folder as this one there is a script called `DutchV1_3_Conversation_Script.py` if not open the `DutchV1_3_Conversation_Script.ipynb` and export it as a .py file.

# Contents:

**1 - Introduction and Overview**

**2 - Setup**
- 2.1 Imports
- 2.2 Open AI
- 2.3 Directories
- 2.4 Previous Files

**3 - Pre Conversation Functions**
- 3.1 Query Screening
- 3.2 Deleting Images
- 3.3 Saving Conversations
- 3.4 Context Check

**4 - Conversation**
- 4.1 - Conversation Interactions and UI
- 4.2 - Conversation Retrieval

# 1 - Introduction and Overview

The DutchessV1 Script works by using a series of Dutch Chatbots to answer user queries. Dutchess first determines under what category according to our question types defined in the Question Types folder the user query falls under. Dutchess then assigns a Dutch, that we specify, to work on that user query and return a response. Dutchess then checks the validity of the response that the Dutch has given and returns a warning about the response if it fails to meet her standards. Essentially Dutchess has several Dutch chatbots working under her that each work to answer a specific type of user query.

# 2 - Setup 

This section details all of the basic peices we need to put together for Dutchess to function, it does not include any functions  but does detail the imports, OpenAI functionality, directories and the loading of the files we set up in the "Resources Creator".

## 2.1 - Imports ##

These are our imports for this notebook, I'll go over what each are used for now:

`from openai import OpenAI` the OpenAI module allows us to set up a client that can communicate with OpenAI's services. These services are not specific to just chatbots although it does include this purpose we can use these services to create vector stores (more on this later) and upload and change files.

`import os` this module allows us to modify and access files and folders

`import pickle` this module allows us to store what cannot be in json files due to information being non-subscriptable

`import json` this module allows for the reading and creation of .json files which allow us to store the data we process for later use

`import numpy as np` this module is for math process'

`import requests` this module allows us to make external requests to outside urls, specifically we will be making requests to OpenAI

`import datetime` this module allows us to retrieve the date and time so we can index our conversations when we save them

`from IPython.display import display, Markdown, clear_output` these imports allow us to display our conversations nicely in the notebook

`from PIL import Image` this module allows for the storage and retreival of images given image data

`import ragas` a module for the assement of our information retrieval process'

`from ragas.metrics import context_precision, context_recall, answer_relevancy` specific measures of our image retrieval process'

`from datasets import Dataset` allows for the processing of data sets we create

`import re` allows us to split text in the conversations we save so the images can be retrieved and show on retirieval of the conversation

In [ ]:
from openai import OpenAI
import os
import pickle
import json
import numpy as np
import requests
import datetime
from IPython.display import display, Markdown, clear_output
from PIL import Image
import ragas
from ragas.metrics import context_precision, context_recall, answer_relevancy
from datasets import Dataset 
import re

If any errors are returned when trying to run the above due to modules not being installed you can remove the # from the appropriate commands below to install the module.

In [ ]:
#pip install openai
#pip install os
#pip install pickle
#pip install json
#pip install numpy
#pip install requests
#pip install datetime
#pip install IPython
#pip install PIL
#pip install ragas
#pip install datasets
#pip install re
#pip install --upgrade ipywidgets

Here we import the conversation script so that we can call the functions that are stored inside of it, if this does not work for any reason please ensure that `DutchV1_3_Conversation_Script.py` exists within the same folder as this notebook, if it does not open the `DutchV1_3_Conversation_Script.ipybn` and export it as a python script.

In [ ]:
import DutchV1_3_Conversation_Script

## 2.2 Open AI ##
`api_key` this is essentially a password provided by OpenAI, it allows us to access OpenAI's services whenever we use them

`client = OpenAI(api_key=api_key)` this sets up a client which can communicate with OpenAI's services, we specify this beforehand so we do not have to write out "OpenAI(api_key=api_key)" when we want to communicate with OpenAI

In [ ]:
api_key = ""

client = OpenAI(api_key=api_key)

## 2.3 Directories ##

We set up any directories for files that we will use later:

`store_name =` this is a general purpose name that we will use when creating files, this allows us to make sure we are retrieving the documents we want later on.

`question_data_directory =`  this is where we retrieve any processing done to the example questions

`conversation_director =` this is where we'll store any conversations we have

`image_directory =` this is the file directory where we'll store and retreieve any images.

In [ ]:
this_directory = os.getcwd()

directories = os.listdir(this_directory)

directories = [os.path.join(this_directory, entry) for entry in directories if not os.path.isfile(os.path.join(this_directory, entry))] 

for directory in directories:
    if "Question Data Store" in os.path.basename(directory):
        question_data_directory = directory
    elif "Conversations" in os.path.basename(directory):
        conversation_directory = directory
    elif "Output Images" in os.path.basename(directory):
        image_directory = directory

print(f"question_data_directory = {question_data_directory}")
print(f"conversation_directory = {conversation_directory}")
print(f"image_directory = {image_directory}")

## 2.4 Previous Files ##

We retrieve the processed question files from the `question_data_directory`

In [ ]:
questions_name = f"questions.json" # creates the file name for the 
questions_path = os.path.join(question_data_directory, questions_name) # creates the file path for the 

with open(questions_path, "r") as file:
    questions = json.load(file) # retrieves our chunks

#-------------------------------------------------------------------------------------------------

question_categories_name = f"questions_categories.json" # creates the file name for the 
question_categories_path = os.path.join(question_data_directory, question_categories_name) # creates the file path for the 

with open(question_categories_path, "r") as file:
    question_categories = json.load(file) # retrieves our chunks

#-------------------------------------------------------------------------------------------------

question_embedding_path = os.path.join(question_data_directory, f"question_embeddings.pkl") # creates the file directory for the pickle file

with open(question_embedding_path, 'rb') as f:
    question_embeddings = pickle.load(f) # retrieves our embedding

# 3 - Pre Conversation Functions

In this section we go over how the user query is screened against the different question types, a function for deleting images uploaded to OpenAI, a function for saving conversations and a function for checking the validity of an answer given by the chatbot.

## 3.1 Query Screening

Query Screening is the process of comparing the given query against types of questions that we have defined in the question types folder. Our function takes an embedding of the query and a threshold value. It then compares the embedding against one of the questions within our question database and if their similairty score is higher than the threshold value the function assigns this score as the new threshold value, this process repeats until the question with the highest similarity score is found. Based on the question that is found the function then returns the analogous question category, if no questions are found that give a score higher than the threshold value no question category is returned. In this process we find the category of a user query by comparing it against pre-existing questions, we can then deal with different queries in different ways

In [ ]:
def query_screening(query_embedding, threshold_value):
    scores = []

    for embedding_data in question_embeddings.data:
        question_embedd = np.array(embedding_data.embedding).flatten()
        scores.append(DutchV1_3_Conversation_Script.cosine_similarity(query_embedding, question_embedd))

    index = len(scores) + 1

    for i in range(0, len(scores)):
        if scores[i] > threshold_value:
            threshold_value = scores[i]
            index = i
    
    if index > len(scores):
        pass

    else:
        return question_categories[index]

## 3.2 Deleting Images

When we want to show an image to the user we also upload the image to OpenAI's servers so that the chatbot we are speaking to can also see the image and provide answers around it. To keep storage low we define the below function to delete these images from OpenAI once a converasation is finished

In [ ]:
def delete_images(files):
    for file in files:
        if file != None:
            API_BASE_URL = f"https://api.openai.com/v1" # url where files are stored
            headers = headers = {
                'Authorization': f'Bearer {api_key}', # our password
                'Content-Type': 'application/json'
            }
            response = requests.delete(f"{API_BASE_URL}/files/{file.id}", headers=headers) # delete the file based on its id
            print(f"{file.id}:", response.status_code, response.text) # tells us if it was sucessful

## 3.3 Saving Conversation

Once we are done with a conversation we want to save the conversation locally so we can easily retrieve it later on. The below function allows us to do this.

In [ ]:
def save_conversation(conversation):
    conversation_name = f"Conversation_{datetime.datetime.now().strftime('%d-%m-%Y_%H-%M')}.json" # specifies a name for the file based on date and time of conversation
    conversation_path = os.path.join(conversation_directory, conversation_name) # specifies where to store the file

    with open(conversation_path, "w") as file:
        json.dump(conversation, file) # saves the conversation
    print(f"conversation saved as {conversation_path}") 

## 3.4 Context Check

The Context Check function takes in the user query, the combined string (which is the immediate knowledge we give the chatbot) and the answer given by the chatbot. It then process' these and checks if the given answer is both relevant to the user query and if it is true to the combined string. It then returns if the answer is fully reliable and relavent as a boolean, returning false if it isnt and returning true if it is.

In [ ]:
def context_check(UserQuery, CombinedString, given_answer):
    os.environ['OPENAI_API_KEY'] = api_key

    DoesDutchKnow = True
    
    ragdataset = {
        'question': [f"{UserQuery}"],
        'answer': [f"{given_answer}"],
        'ground_truth': [CombinedString],
        'contexts': [[f"{CombinedString}"]]
    }  # Setting up our dataset for RAGAS
    
    RagData = Dataset.from_dict(ragdataset)  # Formatting our dataset with all the relevant info into a dataset that can be passed into the RAGAS library
    
    result = ragas.evaluate(
        RagData,
        metrics=[
            context_precision,
            context_recall,
            answer_relevancy
        ]
    )  # Performing the tests on our data to see how relevant the knowledge base is to the given question
    
    print("results found")
    
    result_df = result.to_pandas()
    ContextPrecision = result_df.at[0, 'context_precision']
    ContextRecall = result_df.at[0, 'context_recall']
    AnswerRelevancy = result_df.at[0, 'answer_relevancy']
    
    if ContextPrecision < 0.75 or ContextRecall < 0.5 or AnswerRelevancy < 0.7:
        DoesDutchKnow = False

    return DoesDutchKnow


# 4 - Conversation

In this section we go over the process of setting up a conversation function that a user can interact with and the function nessecary for retrieving previous conversations

## 4.1 Conversation Interactions and UI

Although the below looks complicated it essentially consists of 3 parts: The User Enters the Query, The Processing of that Query, How to respond to the Query. There are two of these 3 parts split by whether it is or is not the first message in the conversation. If it is not the first message in the conversation the function goes to a while loop.

#### The User Enters the Query:
```
user_input = input("Enter your Query")
messages = f"**User**: {user_input} \n"
display(Markdown(messages))
```

this takes the user query, stores that user query inside the `messages` string and then displays the `messages` string, the `messages` string is where we will store all messages sent by both the assistant and user and is what is stored when we store the final conversation, this the same regardless of whether it is or is not the first message in the conversation.

#### The Processing of that Query:
```
#------------------------------------------------------------
if user_input == "quit" or user_input == "Quit" or user_input == "":
    pass
#------------------------------------------------------------

query_embedding = DutchV1_3_Conversation_Script.query_embedd(query=user_input)
category = query_screening(query_embedding, 0.9)

sources = []
combined_string = ""

similarity_scores = DutchV1_3_Conversation_Script.get_similarities(query_embedding=query_embedding)
if len(similarity_scores) != 1 and len(similarity_scores) !=0:
    combined_string, sources = DutchV1_3_Conversation_Script.get_combined_string(similarity_scores=similarity_scores) # gets the best chunks releveant to the query
image_data = DutchV1_3_Conversation_Script.image_retrieval(query=user_input)
#------------------------------------------------------------
```

this first checks if the user query was to quit and if it was it ends the conversation, it then embedds the user query and runs it through the screening process to retrieve the question category, the final part takes the embedding and retireves the relevant information to the query and retrieves the images closest in relevance to the query. This is the part within the "first message" part of the conversation, the non "first message" part only differs by the content of the result of the user inputing quit which looks like this:

```
if user_input == "quit" or user_input == "Quit" or user_input == "": # checks if the user input is quit
    messages += "\n\nQuit Successful"
    clear_output(wait=True)
    display(Markdown(messages))
        if files !=[]:
            delete_images(files)
    save_conversation(messages)
    break
```
this part reprints the history of messages and deletes any image files that were sent to OpenAI, it then saves the conversation and ends the while loop, thus ending the conversation.

#### How to respond to the Query
This is arguably the most important part of the process as it defines the behaviour of your final product. This step takes the category that we found in the processing step and returns a response based upon this step. The response methods are modular so a new one can very easily be added. All of the modules start with an if, elif or else statement to check what category the query was assigned to, the else statement acts as a catch all for any queries not assigned to any categories. Once we have defined the category statements we can call 1 of 5 functions to process the user query. These are from the  `DutchV1_3_Conversation_Script.py` and are `start_conversation`,`manual_assistant_message_start`, `continue_conversation`, `continue_conversation_with_image` and `manual_assistant_message_continue`. However these five can be split into two categories: First Message Functions and Non First Message Functions. The First Message Functions are the `start_conversation` function and the `manual_assistant_message_start` functions and the Non First Message Functions are `continue_conversation`, `continue_conversation_with_image` and `manual_assistant_message_continue`. 

The First Message Functions should only ever be called on the first message that the user sends as such these will only every be called once within a conversation. 

The `start_conversation` function is the function that sends a message to a given assistant and retrieves a response, it is formatted as `start_conversation(prompt,assistant_name, sources, image_data)` where the `prompt` is a processed string usually including the original user query, the combined string and the description of the image found relating to the user query and any instructions on how to deal with the user query that is sent to the assistant, the `assistant_name` is the name of the assistant you want to use for this response, `sources` is the list of sources for the combined string and `image_data` is the image data for the found image relating to the user query, in the below I have used this function in the `if Category == "Capacity.txt` and the `else` statement as for the former I want the assistant to explain about what it can do and in the later I want the assistant to answer general user queries. The `manual_assistant_message_start` function enables us to give predetermined answer to specific user queries, it is formatted as `manual_assistant_message_start(assistant_output)` where the `assistant_output` is the predetermined answer, in the below I have used this for `elif category == "Safety.txt":` and `elif category == "Silly.txt":` as if given a question about safety I want the assistant to not give any specific answers but to defer to an outside source to avoid liability and if the user asks a "silly" question I want the assistant to give a fixed answer to say that it cannot answer that user query.

The Non First message functions are called within the while loop and should only be called after a First Message Function has been called already as the First Message Functions create a `thread` which all of the Non First Message functions take as an input.

The `continue_conversation` function works identically to the `start_conversation` function except it does not take in any image data and does not produce images, below this is used in an identical place as the `start_conversation` function except of course it is within the while loop. The `continue_conversation_with_image` function is a slight modification of the `continue_conversation` function with the added capability of handling images, in the below I have used this for `elif category == "Images.txt":` so that should a user ask for a relevant image to a given explanation they are given one. The reason we seperate 
`continue_conversation` and `continue_conversation_with_image` is because we do not want to be constantly giving the user images every time the assistant responds. The `manual_assistant_message_continue` works identically to the `manual_assistant_message_start` and is used in identical places inside the while loop. The format of these functions are: `continue_conversation(prompt, thread, assistant_name, sources,)`, `continue_conversation_with_image(prompt,thread, assistant_name, sources, image_data)`, `manual_assistant_message_continue(assistant_output, thread)`, the only new variable in these functions is the `thread` which is an output from the First Message Functions and needs to be passed to these functions so that new messages are added to the same conversation. 

The end of each of each of the modules follows the same format:

```
files.append(file)

clear_output(wait=True)
messages  += f"\n **____ Assistant Used**: {assistant_output}"

DoesDutchKnow = context_check(user_input, combined_string, assistant_output)
        if DoesDutchKnow == False:
            messages += f"\n INSERT WARNING MESSAGE HERE"

display(Markdown(messages))   
 
```

`files.append(file)` adds the reference that OpenAI uses for the image to a list so that they can be deleted at the end of the conversation, this line only needs to be included if the function used in the module returns an image (i.e the `start_conversation` and `continue_conversation_with_image` function)

We then clear the previous output and add the new output message to the messages string.

If we need to check the content of the message to ensure it is correct we need to run a context check, if it returns false we append a warning to the messages string.

Finally we display the conversation history again.


Here is a stripped down version of the conversation() function with only 1 module in each part for easier viewing:
```
def conversation():
    files = []
```

The User Enters the Query:
```
    user_input = input("Enter your Query")
    messages = f"**User**: {user_input} \n"
    display(Markdown(messages))
```

The Processing of that Query:
```
    if user_input == "quit" or user_input == "Quit" or user_input == "":
        pass
    
    query_embedding = DutchV1_3_Conversation_Script.query_embedd(query=user_input)
    category = query_screening(query_embedding, 0.9)
    
    sources = []
    combined_string = ""

    similarity_scores = DutchV1_3_Conversation_Script.get_similarities(query_embedding=query_embedding)
    if len(similarity_scores) != 1 and len(similarity_scores) !=0:
        combined_string, sources = DutchV1_3_Conversation_Script.get_combined_string(similarity_scores=similarity_scores)
    image_data = DutchV1_3_Conversation_Script.image_retrieval(query=user_input)
```
How to respond to the Query
```
    if category = `category`:
        if image_data:
            image, description, image_path, image_name = image_data
            prompt = prompt with user_input, combined_string, description of image and instructions on how to deal with them
        else:
            image_data = None
            prompt = prompt with user_input, combined_string and instructions on how to deal with them
        
        assistant_name = "Labs General"
        
        thread, assistant_output, file = DutchV1_3_Conversation_Script.start_conversation(prompt,assistant_name, sources, image_data)
        files.append(file)

        clear_output(wait=True)
        messages  += f"\n **Explanation Assistant Used**: {assistant_output}"

        DoesDutchKnow = context_check(user_input, combined_string, assistant_output)
        if DoesDutchKnow == False:
            messages += f"\n\n WARNING MESSAGE"

        display(Markdown(messages)) 
```
Enter the while loop 
```
    while user_input != "quit" and user_input != "Quit" and user_input != "": 

            #The User Enters the Query:

            user_input = input() # takes another user input
            clear_output(wait=True)
            messages += f"\n\n**User**: {user_input} \n"
```
The Processing of that Query:
```  
            if user_input == "quit" or user_input == "Quit" or user_input == "":
                messages += "\n\nQuit Successful"
                clear_output(wait=True)
                display(Markdown(messages))
                if files !=[]:
                    delete_images(files)
                save_conversation(messages)
                break
           
            query_embedding = DutchV1_3_Conversation_Script.query_embedd(query=user_input)
            category = query_screening(query_embedding, 0.9)

            sources = []
            combined_string = ""

            similarity_scores = DutchV1_3_Conversation_Script.get_similarities(query_embedding=query_embedding)
            if len(similarity_scores) != 1 and len(similarity_scores) !=0:
                combined_string, sources = DutchV1_3_Conversation_Script.get_combined_string(similarity_scores=similarity_scores)
            image_data = DutchV1_3_Conversation_Script.image_retrieval(query=user_input)
```
How to respond to the Query
```
            if category == `category`:
                prompt = prompt with user_input, combined_string and instructions on how to deal with them

                assistant_output = DutchV1_3_Conversation_Script.continue_conversation(prompt, thread, assistant_name, sources,)
        
                clear_output(wait=True)
                messages  += f"\n **Explanation Assistant Used**: {assistant_output}"
                
                DoesDutchKnow = context_check(user_input, combined_string, assistant_output)
                if DoesDutchKnow == False:
                    messages += f" WARNING MESSAGE" 

                display(Markdown(messages)) 
```

In [ ]:
def conversation():
    files = []

    #------------------------------------------------------------
    user_input = input("Enter your Query")
    messages = f"**User**: {user_input} \n"
    display(Markdown(messages))
    #------------------------------------------------------------
    if user_input == "quit" or user_input == "Quit" or user_input == "":
        pass
    #------------------------------------------------------------
    query_embedding = DutchV1_3_Conversation_Script.query_embedd(query=user_input)
    category = query_screening(query_embedding, 0.9)
    
    sources = []
    combined_string = ""

    similarity_scores = DutchV1_3_Conversation_Script.get_similarities(query_embedding=query_embedding)
    if len(similarity_scores) != 1 and len(similarity_scores) !=0:
        combined_string, sources = DutchV1_3_Conversation_Script.get_combined_string(similarity_scores=similarity_scores) # gets the best chunks releveant to the query
    image_data = DutchV1_3_Conversation_Script.image_retrieval(query=user_input)
    #------------------------------------------------------------
    if category == "Capacity.txt":
        image_data, image, description, image_path, image_name = None, None, None, None, None

        prompt = f"This is the user query: {user_input} You are a chatbot working with a series of other chatbots, your job within this group is  to answer questions about the capacity of other chatbots. These chatbots including yourself work as one chatbot all together and so you should answer questions without making reference to other chatbots but instead as though they are you such that on the users end it appears as though you are a single chatbot. The capacity of the other chatbots is to answer questions about the Discovery Skill Module university course, to answer user queries about scheduling regarding when labs take place, the ability to provide relevant images to the user about their queries and to provide resources regarding safety. When you answer a question about the capacity of other chatbots you should include all and only these purposes. "

        assistant_name = "Labs General"

        thread, assistant_output, file = DutchV1_3_Conversation_Script.start_conversation(prompt,assistant_name, sources, image_data) # calls the start conversation function, giving an output and storing a thread
        files.append(file)


        clear_output(wait=True)
        messages  += f"\n **Capacity Assistant Used**: {assistant_output}"
        display(Markdown(messages))    
    #------------------------------------------------------------
    elif category == "Safety.txt":
        assistant_output  = "Please direct any questions regarding safety to the individual risk assesments on the Discovery Skills Module Page on blackboard"

        thread, file = DutchV1_3_Conversation_Script.manual_assistant_message_start(assistant_output)
        files.append(file)

        clear_output(wait=True)
        messages  += f"\n **Safety Response Used**: {assistant_output} "
        display(Markdown(messages)) 
    #------------------------------------------------------------
    elif category == "Silly.txt":
        assistant_output  = "I am afraid I cannot answer that, I'd be happy to assist you with any other queries you have"

        thread, file = DutchV1_3_Conversation_Script.manual_assistant_message_start(assistant_output)
        files.append(file)

        clear_output(wait=True)
        messages  += f"\n **Silly Response Used**: {assistant_output} "
        display(Markdown(messages)) 
    #------------------------------------------------------------
    else:
        if image_data:
            image, description, image_path, image_name = image_data
            prompt = f"Determine if the user query is simple: This is the user query: {user_input}, if it is simple just give a simple answer and ignore the following instructions you should stop reading the prompt at this point if you determined the query to be simple. if it is not simple then give the answer to the user query in a step by step format, do not state in the answer whether the user query is simple or not, respond to the user by providing hints as to what the next step in the process is rather than just outright giving them the answer, you should ask the user a question at the end of every single step that could help them get the next step, you should only go one step at a time per answer never ever more than this, you may give the user hints but again never give them the full answer, this is also some extra context you may need: {combined_string}. You should explain as if i do not have access to this context and any source documents. run any code you create. quote explicitely any equations in latex format. always use $ when writing latex, speak as if you're talking with a student, when you respond to a user question about a step, do not make a new step, instead answer the question they asked then repeat the step they asked the question about. Please be relatively conversational, so you do not need to title questions and hints as such. You should, but not for every step, ask if the user can explain some part of the previous step back to you before you move on to the next step or alternatively ask the user to write code for the next step before you show them the answer but again dont do this for every step. Sometimes ask the user to write code before you write it for them. Explain every single part of every line of any code. You do not need to title the steps. You have been provided an image, here is its description {description}, only mention the image if the user enquires about it. Never Ever give the user the answer in full in a single message. Please write no more than 300 words per message."
        else:
            image_data = None
            prompt = f"Determine if the user query is simple: This is the user query: {user_input}, if it is simple just give a simple answer and ignore the following instructions you should stop reading the prompt at this point if you determined the query to be simple. if it is not simple then give the answer to the user query in a step by step format, do not state in the answer whether the user query is simple or not, respond to the user by providing hints as to what the next step in the process is rather than just outright giving them the answer, you should ask the user a question at the end of every single step that could help them get the next step, you should only go one step at a time per answer never ever more than this, you may give the user hints but again never give them the full answer, this is also some extra context you may need: {combined_string}. You should explain as if i do not have access to this context and any source documents. run any code you create. quote explicitely any equations in latex format. always use $ when writing latex, speak as if you're talking with a student, when you respond to a user question about a step, do not make a new step, instead answer the question they asked then repeat the step they asked the question about. Please be relatively conversational, so you do not need to title questions and hints as such. You should, but not for every step, ask if the user can explain some part of the previous step back to you before you move on to the next step or alternatively ask the user to write code for the next step before you show them the answer but again dont do this for every step. Sometimes ask the user to write code before you write it for them. Explain every single part of every line of any code. You do not need to title the steps. Never Ever give the user the answer in full in a single message. Please write no more than 300 words per message."
        
        assistant_name = "Labs General"
        
        thread, assistant_output, file = DutchV1_3_Conversation_Script.start_conversation(prompt,assistant_name, sources, image_data) # calls the start conversation function, giving an output and storing a thread
        files.append(file)

        clear_output(wait=True)
        messages  += f"\n **Explanation Assistant Used**: {assistant_output}"

        DoesDutchKnow = context_check(user_input, combined_string, assistant_output)
        if DoesDutchKnow == False:
            messages += f"\n\n **WARNING:** Our context checks indicate the content of this message may not be entirely correct, please refer to the source documents to ensure this information is accurate."

        display(Markdown(messages)) 




    #------------------------------------------------------------
    while user_input != "quit" and user_input != "Quit" and user_input != "": # we use a while loop so that the conversation continues until the user enters "quit" or "Quit"
            #------------------------------------------------------------
            user_input = input() # takes another user input
            clear_output(wait=True)
            messages += f"\n\n**User**: {user_input} \n"
            #------------------------------------------------------------
            if user_input == "quit" or user_input == "Quit" or user_input == "": # checks if the user input is quit
                messages += "\n\nQuit Successful"
                clear_output(wait=True)
                display(Markdown(messages))
                if files !=[]:
                    delete_images(files)
                save_conversation(messages)
                break
            #------------------------------------------------------------

            query_embedding = DutchV1_3_Conversation_Script.query_embedd(query=user_input)
            category = query_screening(query_embedding, 0.9)

            sources = []
            combined_string = ""

            similarity_scores = DutchV1_3_Conversation_Script.get_similarities(query_embedding=query_embedding)
            if len(similarity_scores) != 1 and len(similarity_scores) !=0:
                combined_string, sources = DutchV1_3_Conversation_Script.get_combined_string(similarity_scores=similarity_scores) # gets the best chunks releveant to the query
            image_data = DutchV1_3_Conversation_Script.image_retrieval(query=user_input)
            #------------------------------------------------------------
            if category == "Capacity.txt":
                prompt = f"This is the user query: {user_input} You are a chatbot working with a series of other chatbots, your job within this group is  to answer questions about the capacity of other chatbots. These chatbots including yourself work as one chatbot all together and so you should answer questions without making reference to other chatbots but instead as though they are you such that on the users end it appears as though you are a single chatbot. The capacity of the other chatbots is to answer questions about the Discovery Skill Module university course, to answer user queries about scheduling regarding when labs take place, the ability to provide relevant images to the user about their queries and to provide resources regarding safety. When you answer a question about the capacity of other chatbots you should include all and only these purposes. "

                assistant_name = "Labs General"

                assistant_output = DutchV1_3_Conversation_Script.continue_conversation(prompt, thread, assistant_name, sources,) # calls the start conversation function, giving an output and storing a thread
        
                clear_output(wait=True)
                messages  += f"\n **Capacity Assistant Used**: {assistant_output}"
                display(Markdown(messages)) 
            #------------------------------------------------------------
            elif category == "Images.txt":
                
                image_data = DutchV1_3_Conversation_Script.image_retrieval(query=f"{messages}")
                
                if image_data:
                    image, description, image_path, image_name = image_data

                    prompt = f"Here is an image that will be shown to the user, here is its description: {description}, you should provide insight into the image and ask the user questions about the image"

                    assistant_name = "Labs General"

                    assistant_output, file = DutchV1_3_Conversation_Script.continue_conversation_with_image(prompt,thread, assistant_name, sources, image_data)
                    files.append(file)

                    DoesDutchKnow = context_check(user_input, combined_string, assistant_output)

                else:
                    assistant_output = "I apologise there are no relevant images to be shown"
                    
                    DutchV1_3_Conversation_Script.manual_assistant_message_continue(assistant_output, thread)

                clear_output(wait=True)
                messages  += f"\n **Image Assistant Used**: {assistant_output}"

                if DoesDutchKnow == False:
                    messages += f"\n\n **WARNING:** Our context checks indicate the content of this message may not be entirely correct, please refer to the source documents to ensure this information is accurate." 

                display(Markdown(messages)) 
            #------------------------------------------------------------     
            elif category == "Safety.txt":
                assistant_output  = "Please direct any questions regarding safety to the individual risk assesments on the Discovery Skills Module Page on blackboard"

                DutchV1_3_Conversation_Script.manual_assistant_message_continue(assistant_output, thread)

                clear_output(wait=True)
                messages  += f"\n **Safety Response Used**: {assistant_output} "
                display(Markdown(messages)) 
            #------------------------------------------------------------
            elif category == "Silly.txt":
                assistant_output  = "I am afraid I cannot answer that, I'd be happy to assist you with any other queries you have"

                DutchV1_3_Conversation_Script.manual_assistant_message_continue(assistant_output, thread)

                clear_output(wait=True)
                messages  += f"\n **Silly Response Used**: {assistant_output} "
                display(Markdown(messages)) 
            #------------------------------------------------------------
            else:
                prompt = f" Determine if the user query is simple: This is the user query: {user_input}, if it is simple just give a simple answer and ignore the following instructions you should stop reading the prompt at this point if you determined the query to be simple. if it is not simple then give the answer to the user query in a step by step format, do not state in the answer whether the user query is simple or not, respond to the user by providing hints as to what the next step in the process is rather than just outright giving them the answer, you should ask the user a question at the end of every single step that could help them get the next step, you should only go one step at a time per answer never ever more than this, you may give the user hints but again never give them the full answer, this is also some extra context you may need: {combined_string}. You should explain as if i do not have access to this context and any source documents. run any code you create. quote explicitely any equations in latex format. always use $ when writing latex, speak as if you're talking with a student, when you respond to a user question about a step, do not make a new step, instead answer the question they asked then repeat the step they asked the question about. Please be relatively conversational, so you do not need to title questions and hints as such. You should, but not for every step, ask if the user can explain some part of the previous step back to you before you move on to the next step or alternatively ask the user to write code for the next step before you show them the answer but again dont do this for every step. Sometimes ask the user to write code before you write it for them. Explain every single part of every line of any code. You do not need to title the steps. Never Ever give the user the answer in full in a single message. Please write no more than 300 words per message."
                assistant_name = "Labs General" 

                assistant_output = DutchV1_3_Conversation_Script.continue_conversation(prompt, thread, assistant_name, sources,) # calls the start conversation function, giving an output and storing a thread
        
                clear_output(wait=True)
                messages  += f"\n **Explanation Assistant Used**: {assistant_output}"
                
                DoesDutchKnow = context_check(user_input, combined_string, assistant_output)
                if DoesDutchKnow == False:
                    messages += f"\n\n **WARNING:** Our context checks indicate the content of this message may not be entirely correct, please refer to the source documents to ensure this information is accurate." 

                display(Markdown(messages)) 

We then call the conversation function to start the conversation.

In [ ]:
conversation()

## 4.2 Conversation Retrieval

The follow allows you to retireve previous conversations, the first box prints a list of all the names of the conversations in the conversation directory, you can then copy and paste this name into the `loaded_conversation_name =` string and execute the second box to display the conversation and retrieve any images shown to the user in that conversation.  

In [ ]:
conversations = DutchV1_3_Conversation_Script.get_all_files_in_folder(conversation_directory)
i = 1
for conversation in conversations:
    print(f"{i}: {os.path.basename(conversation)}")
    i += 1

In [ ]:
loaded_conversation_name = f"Images.json" 
loaded_conversation_path = os.path.join(conversation_directory, loaded_conversation_name) # gets the path for our descriptions

with open(loaded_conversation_path, "r") as file:
    previous_conversation = json.load(file) # retrieves our descriptions

display(Markdown(previous_conversation))

if "Image Description of" in previous_conversation:
    image_names = re.findall(r"Image Description of ([^:]+):", previous_conversation)
    clean_image_names = [name.strip("*") for name in image_names]
    # Print the extracted image names
    for image_name in clean_image_names:
        image_path = os.path.join(image_directory, image_name)
        Image.open(image_path).show()